<div class="alert alert-block alert-info">
__Name__: heidelburg_train_classifiers<br/>
__Description__: Build and evaluate classifiers for predicting AMR in S. Heidelburg <br/>
__Author__: Matthew Whiteside matthew dot whiteside at canada dot ca<br/>
__Date__: Nov 4, 2017<br/>
__TODO__:<br/>
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from hpsklearn import HyperoptEstimator, random_forest, gradient_boosting, xgboost_classification
from hyperopt import tpe
from sklearn.metrics import f1_score, classification_report
import os
os.chdir('../pangenome')
import utils
import classify
import config

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [35]:
amr = joblib.load(config.SH['amr'])
amr_list = joblib.load(config.SH['amr_list'])
sample_index = joblib.load(config.SH['sample_index'])
pg = joblib.load(config.SH['pg'])
locus_list = joblib.load(config.SH['locus_list'])
test_train_index = joblib.load(config.SH['test_train_index'])

In [36]:
X_train = pg[test_train_index == 'Training',:].toarray()
X_test = pg[test_train_index == 'Validation',:].toarray()
amr_train = amr[test_train_index == 'Training',:]
amr_test = amr[test_train_index == 'Validation',:]
# Define loss function
def loss_fn(y_target, y_prediction):
    return 1.0 - f1_score(y_target, y_prediction)

In [46]:
# AMP
y_train = amr_train[:,amr_list == 'AMP'].flatten()
y_test = amr_test[:,amr_list == 'AMP'].flatten()

# Random forest
rf = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
rf.fit( X_train, y_train )
print(rf.score(X_test, y_test))

rfc = rf._best_learner
print(rfc)
joblib.dump(rfc, config.SH['amp_rfc'])

# Gradient Boosting
gb = HyperoptEstimator( classifier=gradient_boosting('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
gb.fit( X_train, y_train )
print(gb.score(X_test, y_test))

gbc = gb._best_learner
print(gbc)
joblib.dump(gbc, config.SH['amp_gbc'])

# XGBoost
xb = HyperoptEstimator( classifier=xgboost_classification('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
xb.fit( X_train, y_train )
print(xb.score(X_test, y_test))

xbc = xb._best_learner
print(xbc)
joblib.dump(xbc, config.SH['amp_xbc'])


0.880341880342
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=25,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=24, n_jobs=1, oob_score=False, random_state=1,
            verbose=False, warm_start=False)
0.888888888889
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.0025039322018838646, loss='deviance',
              max_depth=3, max_features=0.450875565909173,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=25,
              presort='auto', random_state=2, subsample=0.6253022997311121,
              verbose=0, warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.897435897436
XGBClassifier(base_score=0.5, colsample_bylevel=0.6194499022274127,
       colsample_bytree=0.9308237529344665, gamma=0.006823479489194179,
       learning_rate=0.019820954312667362, max_delta_step=0, max_depth=9,
       min_child_weight=2, missing=nan, n_estimators=800, nthread=-1,
       objective='binary:logistic', reg_alpha=0.29185218878955244,
       reg_lambda=2.477682366207556, scale_pos_weight=1, seed=3,
       silent=True, subsample=0.7458008940987453)


['/media/poolhouse/workspace/l_amr/data/NML/heidelburg_james/ml/amp_xbc.jpkl']

In [55]:
# FOX
y_train = amr_train[:,amr_list == 'FOX'].flatten()
y_test = amr_test[:,amr_list == 'FOX'].flatten()

# Random forest
rf = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, trial_timeout=2000 )
rf.fit( X_train, y_train )
print(rf.score(X_test, y_test))

rfc = rf._best_learner
print(rfc)
joblib.dump(rfc, config.SH['fox_rfc'])

# Gradient Boosting
gb = HyperoptEstimator( classifier=gradient_boosting('gbc'), preprocessing=[], algo=tpe.suggest, trial_timeout=2000 )
gb.fit( X_train, y_train )
print(gb.score(X_test, y_test))

gbc = gb._best_learner
print(gbc)
joblib.dump(gbc, config.SH['fox_gbc'])

# XGBoost
xb = HyperoptEstimator( classifier=xgboost_classification('gbc'), preprocessing=[], algo=tpe.suggest, trial_timeout=2000 )
xb.fit( X_train, y_train )
print(xb.score(X_test, y_test))

xbc = xb._best_learner
print(xbc)
#joblib.dump(xbc, config.SH['fox_xbc'])


0.965811965812
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=0.5204111913706242,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1234, n_jobs=1,
            oob_score=False, random_state=3, verbose=False,
            warm_start=False)
0.957264957265
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.030200196214378496, loss='deviance',
              max_depth=3, max_features=0.8425335506135341,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=11,
              presort='auto', random_state=3, subsample=0.5487092692424331,
              verbose=0, warm_start=False)
0.974358974359
XGBClassifier(base_score=0.5, colsample_bylevel=0.62358

In [48]:
amr_list

array(['AMP', 'FOX', 'STR', 'SOX', 'TCY'],
      dtype='<U3')

In [52]:
# STR
y_train = amr_train[:,amr_list == 'STR'].flatten()
y_test = amr_test[:,amr_list == 'STR'].flatten()

# Random forest
rf = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
rf.fit( X_train, y_train )
print(rf.score(X_test, y_test))

rfc = rf._best_learner
print(rfc)
joblib.dump(rfc, config.SH['str_rfc'])

# Gradient Boosting
gb = HyperoptEstimator( classifier=gradient_boosting('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
gb.fit( X_train, y_train )
print(gb.score(X_test, y_test))

gbc = gb._best_learner
print(gbc)
joblib.dump(gbc, config.SH['str_gbc'])

# XGBoost
xb = HyperoptEstimator( classifier=xgboost_classification('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
xb.fit( X_train, y_train )
print(xb.score(X_test, y_test))

xbc = xb._best_learner
print(xbc)
joblib.dump(xbc, config.SH['str_xbc'])

/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.940170940171
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=16, n_jobs=1, oob_score=False, random_state=1,
            verbose=False, warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.974358974359
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.03667888778055349, loss='exponential',
              max_depth=2, max_features=0.334531506098776,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=232,
              presort='auto', random_state=0, subsample=0.5021421635040079,
              verbose=0, warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.940170940171
XGBClassifier(base_score=0.5, colsample_bylevel=0.788813124016441,
       colsample_bytree=0.6006540590698322, gamma=0.07580224147686597,
       learning_rate=3.86479707267349e-05, max_delta_step=0, max_depth=2,
       min_child_weight=2, missing=nan, n_estimators=3400, nthread=-1,
       objective='binary:logistic', reg_alpha=0.1643066883467893,
       reg_lambda=1.3120245038561862, scale_pos_weight=1, seed=2,
       silent=True, subsample=0.6182551360088798)


['/media/poolhouse/workspace/l_amr/data/NML/heidelburg_james/ml/str_xbc.jpkl']

In [53]:
# SOX
y_train = amr_train[:,amr_list == 'SOX'].flatten()
y_test = amr_test[:,amr_list == 'SOX'].flatten()

# Random forest
rf = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
rf.fit( X_train, y_train )
print(rf.score(X_test, y_test))

rfc = rf._best_learner
print(rfc)
joblib.dump(rfc, config.SH['sox_rfc'])

# Gradient Boosting
gb = HyperoptEstimator( classifier=gradient_boosting('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
gb.fit( X_train, y_train )
print(gb.score(X_test, y_test))

gbc = gb._best_learner
print(gbc)
joblib.dump(gbc, config.SH['sox_gbc'])

# XGBoost
xb = HyperoptEstimator( classifier=xgboost_classification('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
xb.fit( X_train, y_train )
print(xb.score(X_test, y_test))

xbc = xb._best_learner
print(xbc)
joblib.dump(xbc, config.SH['sox_xbc'])

/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.982905982906
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=0.937066001509703,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=132, n_jobs=1,
            oob_score=False, random_state=0, verbose=False,
            warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cla

0.982905982906
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.545108316119107, loss='deviance',
              max_depth=None, max_features=0.8867839114891446,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=4, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=12,
              presort='auto', random_state=4, subsample=0.9253978207816151,
              verbose=0, warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cla

0.957264957265
XGBClassifier(base_score=0.5, colsample_bylevel=0.9368967686733244,
       colsample_bytree=0.9654881785039708, gamma=0.14931291590264104,
       learning_rate=0.0002473003531724035, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=nan, n_estimators=1200, nthread=-1,
       objective='binary:logistic', reg_alpha=0.008975942107309435,
       reg_lambda=1.2306486189020434, scale_pos_weight=1, seed=1,
       silent=True, subsample=0.8121553901490658)


['/media/poolhouse/workspace/l_amr/data/NML/heidelburg_james/ml/sox_xbc.jpkl']

In [54]:
# TCY
y_train = amr_train[:,amr_list == 'TCY'].flatten()
y_test = amr_test[:,amr_list == 'TCY'].flatten()

# Random forest
rf = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
rf.fit( X_train, y_train )
print(rf.score(X_test, y_test))

rfc = rf._best_learner
print(rfc)
joblib.dump(rfc, config.SH['tcy_rfc'])

# Gradient Boosting
gb = HyperoptEstimator( classifier=gradient_boosting('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
gb.fit( X_train, y_train )
print(gb.score(X_test, y_test))

gbc = gb._best_learner
print(gbc)
joblib.dump(gbc, config.SH['tcy_gbc'])

# XGBoost
xb = HyperoptEstimator( classifier=xgboost_classification('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000 )
xb.fit( X_train, y_train )
print(xb.score(X_test, y_test))

xbc = xb._best_learner
print(xbc)
joblib.dump(xbc, config.SH['tcy_xbc'])

/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.794871794872
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=0.1615044840341744,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=21, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=782, n_jobs=1,
            oob_score=False, random_state=2, verbose=False,
            warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.854700854701
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.02206464000085056, loss='exponential',
              max_depth=None, max_features=0.08772963579864812,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=27, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=82,
              presort='auto', random_state=2, subsample=0.7060654225719311,
              verbose=0, warm_start=False)


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.897435897436
XGBClassifier(base_score=0.5, colsample_bylevel=0.5065795158895923,
       colsample_bytree=0.5701108430765363, gamma=0.0012322856981295518,
       learning_rate=0.04950706088782154, max_delta_step=0, max_depth=1,
       min_child_weight=6, missing=nan, n_estimators=3200, nthread=-1,
       objective='binary:logistic', reg_alpha=0.0026330252031758383,
       reg_lambda=1.4031163066674648, scale_pos_weight=1, seed=3,
       silent=True, subsample=0.7488192028231)


['/media/poolhouse/workspace/l_amr/data/NML/heidelburg_james/ml/tcy_xbc.jpkl']